<a href="https://colab.research.google.com/github/SenuDyl/Proxy-Signature-Scheme/blob/main/Proxy_Signatures_Scheme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Proxy Signature Protocol for Partial Delegation

## Greatest Common Divisor

We need this function in the signature scheme as a utility function.

In [ ]:
def gcd(a, b):
    if((a<0) or (b<0) or (a<b)):
        print("wrong parameter input")
        return

    while(b != 0):
        r = a % b
        a = b
        b = r

    return a

In [ ]:
a = 31
b = 23
d = gcd(a,b)
print('gcd(',a,',',b,')=',d)

a = 16
b = 9
d = gcd(a,b)
print('gcd(',a,',',b,')=',d)

a = 468
b = 249
d = gcd(a,b)
print('gcd(',a,',',b,')=',d)

gcd( 31 , 23 )= 1
gcd( 16 , 9 )= 1
gcd( 468 , 249 )= 3


# Protocol

## Setup: Key pair generation of original signer

public parameter - modulus: prime $p$

public parameter - generator for $Z^*_p$: $g$

public key: v

secret key: s

__In this test implementation, only input needs to be given by the user is the prime modulus $p$.__

### Efficiently find a generator $g$ where $p$ is a large prime (e.g. 32 bits)

1.   Allow the use of a large prime number $p$ (for example, a 32 bit number)

2.   Efficiently find a generator $g$ for a multiplicative group of integers modulo $p$

To efficiently find a generator $g$ for a large prime $p$, we use the fact that an element $g$ ∈ $Z^*_p$ is a generator if its order is $p - 1$. By factoring
$p - 1$ and checking that
$$
a^{\frac{p-1}{q}} \not\equiv 1 \pmod{p}
$$
for all prime factors $q$, we can quickly verify if $a$ generates the entire group without testing all powers, making the method efficient even for large primes.
Also, using Python's pow() with three arguments (pow(base, exp, mod)) to do modular arithmetic is quicker than using % by itself.


### Generate a random number of 32-bit and check for its primality using Miller-Rabin primality test

In [ ]:
import math
import random

# Miller–Rabin primality test
def is_prime(n, k=20):
    """Check if n is prime using Miller-Rabin primality test"""
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0:
        return False

    # Write n-1 as 2^r * d
    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2

    # Witness loop
    for _ in range(k):
        a = random.randrange(2, n - 2)
        x = pow(a, d, n)  # a^d % n
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

def get_large_prime(bits=32):
    """Generate a random prime of given bit length."""
    while True:
        # Pick a random number in [2^(bits-1), 2^bits - 1]
        n = random.randrange(2**(bits-1), 2**bits)
        if is_prime(n):
            return n

In [ ]:
print(get_large_prime())

3243519073


In [ ]:
import math
import random

def prime_factors(n):

    """ Returns a set of prime factors of n """
    factors = set()

    # Check divisibility by 2
    while n % 2 == 0:
        factors.add(2)
        n = n // 2

    # Check odd numbers from 3 upwards
    for i in range(3, int(math.sqrt(n)) + 1, 2):
        while n % i == 0:
            factors.add(i)
            n = n // i

    # If remaining n is a prime number greater than 2
    if n > 2:
        factors.add(n)

    return factors

def find_generator_efficient(p):
    # Here we calculate the euler totient of the given prime p
    phi = p - 1
    factors = prime_factors(phi)

    for a in range(2, p):
        for q in factors:
          # Using pow() function to calculate the modulus explicitly increases the efficiency greatly
            if pow(a, phi // q, p) == 1:
                break
        else:
            return a  # found a generator


# p = 4294967311  # Giving 32-bit prime directly
p = get_large_prime() # Get a random prime number using Miller-Rabin primality test
g = find_generator_efficient(p)
s = random.randint(1,p-2) # secret key, Z_{p-1}/{0}
v = pow(g, s, p) # public key

print('*public parameters*')
print('prime modulus (p):',p)
print('generator     (g):',g)
print('*key pair of original signer*')
print('public key    (v):',v)
print('secret key    (s):',s)


*public parameters*
prime modulus (p): 2879745989
generator     (g): 2
*key pair of original signer*
public key    (v): 354663066
secret key    (s): 1525162280


## Step 1: Proxy generation

Proxy: ($\sigma$, K)

In [ ]:
import random
k = random.randint(1,p-2) # random number, Z_{p-1}/{0}
print('(just any) random number k:',k)

print('*proxy value pair for proxy signer*')

bigk = pow(g, k, p)
print('K    :',bigk)

sigma = pow((s + k*bigk), 1, p-1)
print('sigma:',sigma)

(just any) random number k: 16460381
*proxy value pair for proxy signer*
K    : 2204211099
sigma: 177147983


## Step 2: Proxy delivery

The proxy $(\sigma,K)$, must be given by original signer to proxy signer securely.

For example, using Diffie-Hellman to create a secure tunnel, this can be done.

## Step 3: Proxy verification

In [ ]:
lhs = pow(g, sigma, p)

# rhs = (v*(bigk**bigk))%p
rhs = (v * pow(bigk, bigk, p)) % p

print('LHS:',lhs)
print('RHS:',rhs)

if(lhs==rhs):
    print('Proxy Verification: Passed')
else:
    print('Proxy Verification: Failed')

LHS: 1583117598
RHS: 1583117598
Proxy Verification: Passed


## Step 4: Signing by the proxy signer

We are going to use __ElGamal signature scheme__.

The __ElGamal signing__ protocol to sign message $m$ where the secret key of the signer is $x$:
1. Choose an integer $k$ randomly from $\{ 2 \ldots p−2 \}$  with $k$ relatively prime to $p−1$.
2. Compute $r \equiv g^{k} \pmod p$.
3. Compute $s \equiv (H(m)-xr)k^{-1} \pmod {p-1}$.
If s = 0, then start again with a different random k.

The signature is $(r,s)$.

When generating the proxy signature, use $\sigma$ instead of the secret key $x$.

### a. Upload file



In [ ]:
from google.colab import files
uploaded = files.upload()

fileName = list(uploaded.keys())[0]
print("Uploaded file:", fileName)

# Open and read the file
with open(fileName, "rb") as f:
    content = f.read()

print("File content:\n")
print(content)

Saving SampleMessage.txt to SampleMessage.txt
Uploaded file: SampleMessage.txt
File content:

b'This is a secret message'


### b. Hash the file content

In [ ]:
import hashlib

# Hash the file content
file_hash = hashlib.sha256(content).hexdigest()

# Convert the file hash to a integer
m = int(file_hash, 16) % (p-1)  # Ensure that 1 <= m <= p-1

print("Message (hashed from file):", m)

Message (hashed from file): 923845338


In [ ]:
k = 0

for i in range (1000): # this number 1000 is just a random value for the number of tries
    k = random.randint(2,p-2) # random number k is in {2 ... (p-2)} and gcd(k,(p-1))=1
    #print('testing',i,'th random number k:',k)

    d = gcd(p-1,k)
    if(d==1):
        break

print('(a special) random number k:',k,'(found after',i,'tries. This is relatively prime to p-1 =',p-1,')')

r = pow(g, k, p)
print('r :',r)

s1 = m - (sigma*r) # in place of s, use sigma, theproxy secret
print('s1:',s1)

(a special) random number k: 1248979951 (found after 1 tries. This is relatively prime to p-1 = 2879745988 )
r : 2365268700
s1: -419002578534186762


### Efficient method for finding modulo inverse k mod q

In [ ]:
def modinv(a, m):
    # Extended Euclidean Algorithm
    m0, x0, x1 = m, 0, 1
    if m == 1:
        return 0
    while a > 1:
        q = a // m
        a, m = m, a % m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1

# Example
invk = modinv(k, q)
print('Modulo inverse of k mod q:',invk,'(for',k,'mod',q,')')

Modulo inverse of k mod q: 1045782651 (for 1248979951 mod 2879745988 )


In [ ]:
s = (s1*invk)%(p-1)
print('s:',s)
print('The proxy signed message is (m,(r,s),K):(',m,'(',r,',',s,')',bigk,')')

s: 2243452990
The proxy signed message is (m,(r,s),K):( 923845338 ( 2365268700 , 2243452990 ) 2204211099 )


In [ ]:
proxy_signature = {
    "message": m,
    "r": r,
    "s": s,
    "K": bigk
}

file_path = "proxy_signature.txt"

with open(file_path, "w") as f:
  f.write("Proxy Signed Message:\n")
  f.write(f"m: {proxy_signature['message']}\n")
  f.write(f"r: {proxy_signature['r']}\n")
  f.write(f"s: {proxy_signature['s']}\n")
  f.write(f"K: {proxy_signature['K']}\n")

print(f"Proxy signature written to {file_path}")

Proxy signature written to proxy_signature.txt


In [ ]:
with open(file_path, "r") as f:
  proxy_signature = f.read()

print("File content:\n")
print(proxy_signature)

File content:

Proxy Signed Message:
m: 923845338
r: 2365268700
s: 2243452990
K: 2204211099



## Step 5: Verification of the proxy signature

The new public key $v' \equiv vK^K \pmod p$

The __ElGamal signature verification__ protocol to verify a signed message $m$ with signature $(r,s)$ where the public key of the signer is $y$:
1. Verify that $0 < r < p$ and $0 < s < p − 1$.
2. Verify that $g^{H(m)} \equiv y^r r^s \pmod p$.

In [ ]:
# Read from disk
data = {}

with open("proxy_signature.txt", "r") as f:
    for line in f:
        line = line.strip()
        if line == "" or line.startswith("Proxy Signed Message"):
            continue
        key, value = line.split(":", 1)  # split only at the first colon
        data[key.strip()] = int(value.strip())  # remove spaces

# Extract values
m = data["m"]
r = data["r"]
s = data["s"]
bigk = data["K"]

print("m:", m, "r:", r, "s:", s, "K:", bigk)


m: 923845338 r: 2365268700 s: 2243452990 K: 2204211099


In [ ]:
print('r =',r,', s =',s,', p =',p)
check1 = False
check2 = False
# Verification Check 1
if (0<r) & (r<p):
    if (0<s) & (s<(p-1)):
        check1 = True
        print('Verification Check 1: Passed')
    else:
        print('Verification Check 1: Failed')
else:
    print('Verification Check 1: Failed')

r = 2365268700 , s = 2243452990 , p = 2879745989
Verification Check 1: Passed


In [ ]:
# Verification check 2
y = rhs # this is the proxy public key y = VK^K mod p

lhs = pow(g, m, p)

rhs = (pow(y, r, p) * pow(r, s, p)) % p

print('LHS:',lhs)
print('RHS:',rhs)

if(lhs==rhs):
    check2 = True
    print('Verification Check 2: Passed')
else:
    print('Verification Check 2: Failed')

LHS: 1426073877
RHS: 1426073877
Verification Check 2: Passed


In [ ]:
if(check1 & check2):
    print('Proxy signature verification: Passed')
else:
    print('Proxy signature verification: Failed')

Proxy signature verification: Passed
